In [1]:
import os
import json

try:
    with open("config.json") as f:
        config = json.load(f)
    os.environ["GOOGLE_API_KEY"] = config["API_KEY"]
    print("Loaded API key")
except Exception as e:
    print("Config/API Key Error:", e)

try:
    from langchain_google_genai import ChatGoogleGenerativeAI
    from langchain_google_genai import GoogleGenerativeAIEmbeddings
    from langchain.vectorstores import Chroma
    from langchain.text_splitter import CharacterTextSplitter
    from langchain.chains import RetrievalQA
    print("Imports successful")
except Exception as e:
    print("Import Error:", e)

try:
    documents = [
        "This is a sample document about RAG.",
        "RAG combines retrieval and generation for better answers.",
        "LangChain is a popular framework for building RAG systems.",
        "Here is the epsilon-delta definition of a limit: $$\lim_{x \to a} f(x) = L \iff \forall \epsilon > 0, \exists \delta > 0 \text{ such that } 0 < |x - a| < \delta \implies |f(x) - L| < \epsilon$$"
    ]
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.create_documents(documents)
    print("Text splitting successful")
except Exception as e:
    print("Text Split Error:", e)

try:
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    print("Embeddings object created")
except Exception as e:
    print("Embeddings Error:", e)

try:
    from langchain.vectorstores import FAISS
    vectorstore = FAISS.from_documents(texts, embeddings)
    print("Vectorstore created")
except Exception as e:
    print("Vectorstore Error:", e)

try:
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())
    print("QA chain created")
except Exception as e:
    print("QA Chain Error:", e)

try:
    queries = [
        "State the definition of a limit (in words)", 
        "State the epsilon-delta definition of a limit for $f(x) = \sin(\frac{1}{x})$ as $x \to 0$", 
        "State the sequential definition and neighborhood definitions of a limit"
    ]
    for query in queries:
        result = qa_chain.run(query)
        print("Query:", query)
        print("Result:", result)
except Exception as e:
    print("Query Error:", e)

Loaded API key
Imports successful
Text splitting successful
Embeddings object created
Vectorstore created
QA chain created


C:\Users\evanc\AppData\Local\Temp\ipykernel_121300\3656116074.py:62: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(query)


Query: State the definition of a limit (in words)
Result: The limit of f(x) as x approaches a is L if and only if for all epsilon greater than 0, there exists a delta greater than 0 such that 0 < |x - a| < delta implies |f(x) - L| < epsilon.
Query: State the epsilon-delta definition of a limit for $f(x) = \sin(rac{1}{x})$ as $x 	o 0$
Result: The epsilon-delta definition of a limit is: $$\lim_{x 	o a} f(x) = L \iff orall \epsilon > 0, \exists \delta > 0 	ext{ such that } 0 < |x - a| < \delta \implies |f(x) - L| < \epsilon$$

For $f(x) = \sin(\frac{1}{x})$ as $x \to 0$, the epsilon-delta definition of a limit is:
$$\lim_{x \to 0} \sin(\frac{1}{x}) = L \iff \forall \epsilon > 0, \exists \delta > 0 \text{ such that } 0 < |x - 0| < \delta \implies |\sin(\frac{1}{x}) - L| < \epsilon$$
Query: State the sequential definition and neighborhood definitions of a limit
Result: I am sorry, but the provided context only includes the epsilon-delta definition of a limit. It does not include the seque

In [ ]:
# Cell: Implement Agentic RAG using Gemini and LangChain's AgentExecutor
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.schema import AgentAction, AgentFinish
import re

# Define a custom prompt template for the agent
from typing import ClassVar

class CustomPromptTemplate(StringPromptTemplate):
    template: ClassVar[str] = """Answer the following question as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

    def format(self, **kwargs):
        # Get the intermediate steps (AgentAction, Observation tuples)
        intermediate_steps = kwargs.pop("agent_scratchpad")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in kwargs["tools"]])
        kwargs["tool_names"] = ", ".join([tool.name for tool in kwargs["tools"]])
        return self.template.format(**kwargs)

from langchain.agents.agent import AgentOutputParser

# Define a custom output parser that inherits from AgentOutputParser
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output):
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        regex = r"Action: (.*?)[\n]*Action Input:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            return AgentFinish(
                return_values={"output": "I could not determine the next action."},
                log=llm_output,
            )
        action = match.group(1).strip()
        action_input = match.group(2).strip(" ").strip('"')
        return AgentAction(tool=action, tool_input=action_input, log=llm_output)

# Define a tool for document retrieval (using the Gemini-based qa_chain)
def retrieve_documents(query):
    return qa_chain.run(query)

tools = [Tool(name="RetrieveDocuments", func=retrieve_documents, description="Useful for retrieving documents related to the query.")]

# Set up the agent with Gemini
prompt = CustomPromptTemplate(template=CustomPromptTemplate.template, input_variables=["input", "agent_scratchpad", "tools"])
output_parser = CustomOutputParser()
llm_chain = LLMChain(llm=llm, prompt=prompt)  # Using the Gemini llm from the previous cell
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    allowed_tools=[tool.name for tool in tools],
    output_parser=output_parser,
    stop=["\nObservation:"]
)
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

# Test the agentic RAG system with Gemini
query = "What is RAG?"
result = agent_executor.run(query)
print("Query:", query)
print("Result:", result)



> Entering new AgentExecutor chain...


ValueError: Missing some input keys: {'tools', 'input'}